In [1]:
# py310
from model.srnc import SequentialRadiusNeighborsClassifier
from model.rejection import classification_rejection_v2

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad
from anndata import AnnData
import torch
from sklearn.metrics import accuracy_score,adjusted_rand_score,f1_score,precision_score,recall_score
# from mars.model.mars import MARS
# from mars.experiment_dataset import ExperimentDataset
# from args_parser import get_parser

# Setting parameters
predictive_alg = "lightGBM"
embedded_option = "PCA"
shrink_parameter = 1
proportion_unknown = 0.2
control_neighbor = 5
threshold_rejection = 0.8
filter_proportion = 5
data_name = "bench"

/home/vanhoan310/miniconda3/envs/py310/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
# # Data preprocessing
# # Note: change the path to dataset
# adata_raw = sc.read_h5ad('/data/hoan/project24/SemiSupervisedLearning/Data/bench/cellbench.h5ad')
# adata_raw.layers['counts'] = adata_raw.X
# adata_raw

# label_key = 'ground_truth'
# batch_key = 'experiment'
# adata = adata_raw.copy()

In [3]:
# # note that this collection of batches is already intersected on the genes
# adata_raw = sc.read(
#     "data/pancreas.h5ad",
#     backup_url="https://www.dropbox.com/s/qj1jlm9w10wmt0u/pancreas.h5ad?dl=1",
# )
# adata_raw.write('/data/hoan/project24/SemiSupervisedLearning/Data/pancreas.h5ad')
adata_raw = sc.read_h5ad('/data/hoan/project24/SemiSupervisedLearning/Data/pancreas.h5ad')

In [4]:
label_key = 'ground_truth'
batch_key = 'experiment'
adata = adata_raw.copy()
adata.obs[label_key] = adata.obs['celltype']
adata.obs[batch_key] = adata.obs['batch']

In [5]:
adata.shape

(14693, 2448)

In [6]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.svm import LinearSVC
import numpy as np

def classification_rejection_v2(X, Y, Y_all_labels, X_train, Y_train,X_test,predictive_alg, threshold_rejection):
    if predictive_alg == "svm":
        clf = svm.SVC(probability=True, max_iter=100000).fit(X_train, Y_train)
    if predictive_alg == "LinearSVM":
        svc = LinearSVC()
        clf = CalibratedClassifierCV(svc, cv=sss).fit(X_train, Y_train)
        # fix to 5 fold instead of 10 fold ?
    if predictive_alg == "lr":
        clf = LogisticRegression(max_iter=10000).fit(X_train, Y_train)
    if predictive_alg == "lda":
        clf = LinearDiscriminantAnalysis().fit(X_train, Y_train)
    if predictive_alg == "dt":
        clf = DecisionTreeClassifier().fit(X_train, Y_train)
    if predictive_alg == "lightGBM":
        clf = LGBMClassifier(verbose=-1).fit(X_train, Y_train)
    if predictive_alg == "GaussianNB":
        clf = GaussianNB().fit(X_train, Y_train)

    probs_max = [np.max(x) for x in clf.predict_proba(X_test)]
    Y_predict = list(clf.predict(X_test))
    Y_predict_rejection = [Y_predict[i] if probs_max[i] >= threshold_rejection else int(10*len(Y_all_labels) + 1)  for i in range(X_test.shape[0])]
    return  Y_predict_rejection

from scnym2.api import scnym_api
import scanpy as sc
import scnym2
config = scnym2.api.CONFIGS["new_identity_discovery"]
config["n_epochs"] = 10
# increase the weight of the domain adversary 0.1 -> 0.3
config["ssl_kwargs"]["dan_max_weight"] = 0.3
config['batch_size'] = 100

def run_scNym(annotated, unannotated):
    ## scNym
    annotated.X = annotated.X-annotated.X.min()+1.0
    sc.pp.normalize_total(annotated, target_sum=1e6)  # scale each cell to 1e6 total counts
    sc.pp.log1p(annotated)  
    scnym_api(
        adata=annotated,
        task='train',
        groupby='ground_truth',
        out_path='scnym_output',
        config=config
    )
    unannotated.X = unannotated.X-unannotated.X.min()+1
    sc.pp.normalize_total(unannotated, target_sum=1e6)  # scale each cell to 1e6 total counts
    sc.pp.log1p(unannotated)  
    scnym_api(
        adata=unannotated,
        task='predict',
        key_added='scNym',
        trained_model='scnym_output',
        out_path='scnym_output',
        config=config,
    )
    Y_predict_rejection = list(unannotated.obs['scNym'])
    return Y_predict_rejection

tensorflow is not installed, assuming tensorboard is independent


In [7]:
# batch-aware feature selection
sc.pp.highly_variable_genes(adata,n_top_genes=2000, flavor="cell_ranger", batch_key=batch_key)

# use selected genes for integration
adata_hvg = adata[:,adata.var["highly_variable"]].copy()

# run pca
sc.pp.pca(adata_hvg, n_comps=50)
adata_hvg.obsm["X_pca"] = adata_hvg.obsm["X_pca"]

# run combat integration to correct for sample and condition effects
sc.pp.combat(adata_hvg, key=batch_key)

In [8]:
adata_hvg

AnnData object with n_obs × n_vars = 14693 × 2000
    obs: 'celltype', 'sample', 'n_genes', 'batch', 'n_counts', 'louvain', 'ground_truth', 'experiment'
    var: 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'celltype_colors', 'louvain', 'neighbors', 'pca', 'sample_colors', 'hvg'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [9]:
adata_hvg.obs['ground_truth'] = adata_hvg.obs['ground_truth'].astype('category')
adata_hvg.obs['ground_truth'] = adata_hvg.obs['ground_truth'].cat.codes

In [16]:
import time
runtime_vec = []
n_total = adata.n_obs
predictive_alg = 'lightGBM'
# Sample data
for n_subsample in [1000, 2000, 5000, 10000, n_total]:
    start = time.time()
    idx_train = np.random.choice(n_total, n_subsample, replace=False)
    idx_test = np.random.choice(n_total, n_subsample, replace=False)
    
    # Subsample the AnnData
    # adata_sub = adata[idx, :].copy()
    
    exp_10x = adata_hvg[idx_train, :].copy()
    # X_exp_10x = exp_10x.X
    X_exp_10x = exp_10x.obsm["X_pca"]
    y_exp_10x = np.array(exp_10x.obs['ground_truth'], dtype=np.int64)
    
    exp_celseq2 = adata_hvg[idx_test, :].copy()
    # X_celseq2 = exp_celseq2.X
    X_celseq2 = exp_celseq2.obsm["X_pca"]
    y_celseq2 = np.array(exp_celseq2.obs['ground_truth'], dtype=np.int64)
    
    # Loop to remove label randomly
    # for i in np.unique(y_exp_10x):
    remove_label = y_exp_10x[0]
    X_10x = X_exp_10x[y_exp_10x != remove_label]
    y_10x = y_exp_10x[y_exp_10x != remove_label]
    
    print("Data experiment...")
    print(f"Class removed from training:{remove_label}")
    
    
    # annotated_data = ExperimentDataset(X_10x.toarray(), exp_10x.obs_names, exp_10x.var_names, '10x', y_10x)
    # unannotated_data = ExperimentDataset(X_celseq2.toarray(), exp_celseq2.obs_names, exp_celseq2.var_names, 'celseq2', y_celseq2)
    # pretrain_data = ExperimentDataset(X_celseq2.toarray(), exp_celseq2.obs_names, exp_celseq2.var_names, 'celseq2')
    # n_clusters = len(np.unique(unannotated_data.y))
    
    # Create AnnData object and add value from dataframe
    
    annotated = ad.AnnData(X=X_10x)
    annotated.obs['ground_truth'] = y_10x
    annotated.obs['experiment'] = '10x'
    
    unannotated = ad.AnnData(X=X_celseq2)
    unannotated.obs['ground_truth'] = y_celseq2
    unannotated.obs['experiment'] = 'celseq2'
    
    annotated.obs_names = [f"Cell_{i:d}" for i in range(annotated.n_obs)]
    annotated.var_names = [f"Gene_{i:d}" for i in range(annotated.n_vars)]
    
    unannotated.obs_names = [f"Cell_{i:d}" for i in range(unannotated.n_obs)]
    unannotated.var_names = [f"Gene_{i:d}" for i in range(unannotated.n_vars)]
    
    annotated_y = np.array(annotated.obs['ground_truth'], dtype=np.int64)
    annotated_x = np.asarray(annotated.X)
    
    # #  MARS implementation
    # annotated_set = ExperimentDataset(annotated_x, annotated.obs_names, annotated.var_names, data_name, annotated_y)
    
    unannotated_y = np.array(unannotated.obs['ground_truth'], dtype=np.int64)
    unannotated_x = np.asarray(unannotated.X)
    
    # unannotated_set = ExperimentDataset(unannotated_x, unannotated.obs_names, unannotated.var_names, data_name, unannotated_y)
    
    
    # pretrain_data = ExperimentDataset(unannotated_x, unannotated.obs_names, unannotated.var_names, data_name)
    
    # n_clusters=len(np.unique(unannotated_data.y))
    # print(n_clusters)
    # mars = MARS(n_clusters, params, [annotated_set], unannotated_set, pretrain_data, hid_dim_1=1000, hid_dim_2=100)
    # adata, landmarks, scores = mars.train(evaluation_mode=True, save_all_embeddings=False)
    # print(adata)
    # scores
    
    
    # SRNC and Rejection implementation
    data_embbed_x = np.concatenate([annotated_x,unannotated_x])
    data_embbed_y = np.concatenate([annotated_y,unannotated_y])
    y_all_labels = list(set(data_embbed_y))
    
    
    Y_predict_srnc=SequentialRadiusNeighborsClassifier(data_embbed_x, y_all_labels, annotated_x, unannotated_x, annotated_y, predictive_alg,
                                            control_neighbor, shrink_parameter, filter_proportion, threshold_rejection)
    
    
    # Y_predict_rejection=classification_rejection_v2(data_embbed_x,data_embbed_y,y_all_labels,annotated_x,annotated_y,unannotated_x,predictive_alg,threshold_rejection)
    # predictive_alg = 'scNym'
    # Y_predict_rejection=run_scNym(annotated, unannotated)
    print(annotated_x.shape, unannotated_x.shape)
    print('Done')
    end = time.time()
    runtime_vec.append((end-start)/60.0)
    print(runtime_vec)

Data experiment...
Class removed from training:5
(784, 50) (1000, 50)
Done
[0.2671634554862976]
Data experiment...
Class removed from training:4
(1384, 50) (2000, 50)
Done
[0.2671634554862976, 0.3791256586710612]
Data experiment...
Class removed from training:18
(4942, 50) (5000, 50)
Done
[0.2671634554862976, 0.3791256586710612, 0.7298123160998027]
Data experiment...
Class removed from training:12
(9599, 50) (10000, 50)
Done
[0.2671634554862976, 0.3791256586710612, 0.7298123160998027, 1.5031184911727906]
Data experiment...
Class removed from training:5
(11339, 50) (14693, 50)
Done
[0.2671634554862976, 0.3791256586710612, 0.7298123160998027, 1.5031184911727906, 2.808844629923503]


In [17]:
runtime_vec

[0.2671634554862976,
 0.3791256586710612,
 0.7298123160998027,
 1.5031184911727906,
 2.808844629923503]